In [5]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import cross_val_score, cross_val_predict
import plotly.express as px


In [4]:
lista = [1,2,3,3]
set1 = set(lista)

set1.

{1, 2, 3}

Intuito aqui é relembrar pq as moléculas 2-2-Difluoropropane e 1-1-Dichloroethylene

In [6]:
df_pol_raw = pd.read_csv("../data/raw/polar_molecules.CSV")
df_pol_processed = pd.read_csv("../data/processed/processed_polar.csv")
df_pol_raw.iloc[[27,33]]

,Molecule,Formula,Molecule Type,Ei,Alpha,AlphaB,axx,ayy,azz,Dipole,pi_bond,Expt
27,2-2-Difluoropropane,C3H6F2,Other,11.42,6.1100,NaN,NaN,NaN,NaN,2.50,0,25.0
33,1-1-Dichloroethylene,C2H2Cl2,Chlorated,9.81,8.0926,8.0926,9.549,9.153,5.576,1.75,1,30.0


In [7]:
df_all_pol = pd.concat((df_pol_processed, df_pol_raw.iloc[[27,33]]), axis = 0, join = "inner")
df_all_pol.reset_index(inplace=True, drop=True)

In [8]:
df_all_pol.tail()

,Molecule,Formula,Ei,Alpha,Dipole,pi_bond,Expt
33,Trichloroethylene,C2HCl3,9.46,10.2000,1.10,1,50.0
34,1-Chloropropane,C3H7Cl,10.81,8.2110,2.54,0,98.0
35,2-Chloropropane,C3H7Cl,10.79,8.2110,2.57,0,113.0
36,2-2-Difluoropropane,C3H6F2,11.42,6.1100,2.50,0,25.0
37,1-1-Dichloroethylene,C2H2Cl2,9.81,8.0926,1.75,1,30.0


In [17]:
# tentando entender onde se encontram essas moléculas
def color_map(molecule):
    return 'red' if molecule in ['2-2-Difluoropropane', '1-1-Dichloroethylene'] else 'blue'

px.scatter_matrix(df_all_pol, dimensions=["Ei", "Alpha", "Dipole", "pi_bond", "Expt"], 
                  color="Molecule",
                  color_discrete_map={m: color_map(m) for m in df_all_pol["Molecule"]})

# Constructing models

In [10]:
models_list = pickle.load(open("../models/polar/all/all_molecules_models.sav", "rb"))

In [11]:
X = df_all_pol[["Ei", "Alpha", "Dipole", "pi_bond"]]
y = df_all_pol["Expt"]

model_names = ["poly", "svr", "xgb", "nn", "ensemble"]
dict_of_performances = {}
dict_of_predicts = {}

for i, j in enumerate(models_list):
    dict_of_performances[model_names[i]] = -cross_val_score(j, X, y, 
                scoring="neg_mean_absolute_percentage_error", cv = len(X), n_jobs = 6)
    
    dict_of_predicts[model_names[i]] = cross_val_predict(j, X, y, cv = len(X), n_jobs = 6)


In [12]:
df_performances = np.round(pd.DataFrame(dict_of_performances), 2)
df_performances["erro medio"] = df_performances.mean(axis=1)

df_predicts = np.round(pd.DataFrame(dict_of_predicts)).astype(int)

In [13]:
df_performances = pd.concat([df_all_pol[["Molecule", "Formula"]], df_performances, df_all_pol["Expt"]], axis = 1)
df_predicts = pd.concat([df_all_pol[["Molecule", "Formula"]], df_predicts, df_all_pol["Expt"]], axis = 1)

In [14]:
df_predicts.iloc[[37,36,32,28,33]]

,Molecule,Formula,poly,svr,xgb,nn,ensemble,Expt
37,1-1-Dichloroethylene,C2H2Cl2,104,103,71,89,92,30.0
36,2-2-Difluoropropane,C3H6F2,52,61,68,72,63,25.0
32,cis-1-2-Dichloroethylene,C2H2Cl2,121,126,93,126,117,66.0
28,1-Fluorohexane,C6H13F,139,154,148,121,140,80.0
33,Trichloroethylene,C2HCl3,84,41,116,79,80,50.0


In [19]:
df_performances.sort_values("erro medio", ascending=False)

,Molecule,Formula,poly,svr,xgb,nn,ensemble,erro medio,Expt
37,1-1-Dichloroethylene,C2H2Cl2,2.45,2.44,1.37,1.98,2.06,2.060,30.0
36,2-2-Difluoropropane,C3H6F2,1.10,1.44,1.70,1.88,1.53,1.530,25.0
32,cis-1-2-Dichloroethylene,C2H2Cl2,0.83,0.91,0.41,0.91,0.77,0.766,66.0
28,1-Fluorohexane,C6H13F,0.73,0.92,0.85,0.51,0.75,0.752,80.0
33,Trichloroethylene,C2HCl3,0.67,0.19,1.31,0.58,0.60,0.670,50.0
4,Chloroform,CHCl3,0.58,0.89,0.51,0.35,0.58,0.582,50.0
2,Ethyl Acetate(conformer1),C4H8O2,0.40,0.51,0.51,0.37,0.45,0.448,160.0
20,Acetaldehyde,C2H4O,0.31,0.16,0.91,0.34,0.43,0.430,88.0
27,1-Fluorobutane,C4H9F,0.15,0.49,0.72,0.33,0.42,0.422,70.0
23,Methanol,CH4O,0.09,0.74,0.31,0.90,0.06,0.420,20.0


In [18]:
df_all_pol.sort_values("Expt")

,Molecule,Formula,Ei,Alpha,Dipole,pi_bond,Expt
23,Methanol,CH4O,10.85,3.1300,1.67,0,20.0
22,Chloromethane,CH3Cl,11.22,4.2800,2.06,0,25.0
36,2-2-Difluoropropane,C3H6F2,11.42,6.1100,2.50,0,25.0
37,1-1-Dichloroethylene,C2H2Cl2,9.81,8.0926,1.75,1,30.0
31,Dichloromethane,CH2Cl2,11.32,6.3530,2.01,0,32.0
15,1-Fluoropropane,C3H7F,11.30,6.0900,2.07,0,35.0
17,Bromomethane,CH3Br,10.54,5.2600,1.99,0,40.0
19,Ethanol,C2H6O,10.43,4.9500,1.72,0,43.0
33,Trichloroethylene,C2HCl3,9.46,10.2000,1.10,1,50.0
4,Chloroform,CHCl3,11.37,8.1900,1.13,0,50.0
